## Setup Only for Colab

In [ ]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/Colab\ Notebooks/hidden_mediators

In [ ]:
%ls

In [ ]:
from IPython.display import clear_output

In [ ]:
import time
!pip install -r requirements.txt
time.sleep(2)
clear_output()

In [ ]:
import time
# replace `develop` with `install` if you wont make library code changes
!python setup.py develop
time.sleep(2)
clear_output()
# Restart the session after running this

In [ ]:
%cd /content/drive/MyDrive/Colab\ Notebooks

## Semi-Synthetic Generation

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats
from joblib import Parallel, delayed
from proximalde.ukbb_proximal import ProximalDE_UKBB, residualizeW_ukbb
from proximalde.proximal import ProximalDE, residualizeW, svd_critical_value
from proximalde.utilities import covariance
from proximalde.ukbb_data_utils import *
from proximalde.gen_data import gen_data_with_mediator_violations, gen_data_no_controls_discrete_m, gen_data_no_controls, gen_data_complex

In [ ]:
D_label = 'Obese'
Y_label = 'back'
W, _, W_feats, X, X_binary, X_feats, Z, Z_binary, Z_feats, Y, D = load_ukbb_data(D_label=D_label, Y_label=Y_label)

est = ProximalDE_UKBB(model_regression='linear', 
                      binary_D=False,
                      semi=True, cv=3, verbose=1, random_state=3)

_, Zres, Xres, Yres = residualizeW_ukbb(W, D, Z, X, Y, D_label=D_label, Y_label=Y_label, save_fname_addn='', random_state=3, cv=3)[:4] 


In [ ]:
from proximalde.gen_data import SemiSyntheticGenerator

generator = SemiSyntheticGenerator(split=True)
generator.fit(W, D, Z, X, Y, ZXYres=[Zres, Xres, Yres],propensity=np.load('propensity.npy'))
# np.save('propensity.npy', generator.propensity_)


In [ ]:
# np.save('propensity.npy', generator.propensity_)
idx=np.random.choice(np.arange(len(generator.Xepsilon_)),size=50000).astype(int)

In [ ]:
import seaborn as sns 
sns.histplot(generator.Xepsilon_[idx,:5])

In [ ]:
import seaborn as sns 
sns.histplot(generator.Zepsilon_[idx,:5])

In [ ]:
nsamples = 50000
a = 1.0  # a*b is the indirect effect through mediator
b = 1.0
c = .5  # this is the direct effect we want to estimate
d = .0  # this can be zero; does not hurt
e = 1.0  # if the product of e*f is small, then we have a weak instrument
f = 1.0  # if the product of e*f is small, then we have a weak instrument
g = .0  # this can be zero; does not hurt
Wtilde, Dtilde, _, Ztilde, Xtilde, Ytilde = generator.sample(nsamples, a, b, c, g, replace=True)

In [ ]:
np.save('Wtilde.npy', Wtilde)
np.save('Dtilde.npy', Dtilde)
np.save('Ztilde.npy', Ztilde)
np.save('Xtilde.npy',Xtilde)
np.save('Ytilde.npy',Ytilde)

In [ ]:
sns.heatmap(covariance(Z, X))
plt.show()
sns.heatmap(covariance(Ztilde, Xtilde))

In [ ]:
sns.heatmap(covariance(Z, Z))
plt.show()
sns.heatmap(covariance(Ztilde, Ztilde))


In [ ]:

num_plots=10
idx=np.random.choice(np.arange(Z.shape[1]),size=50000).astype(int)
fig, axes = plt.subplots(1, num_plots, figsize=(num_plots*4, 4))
for i in range(num_plots):
    axes[i].hist(Ztilde[:, i], bins=30, alpha=0.5, label=f'sampled', color='blue',density=True)
    axes[i].hist(Z[:, i], alpha=0.5, label=f'true', color='orange',density=True)


    # Add legend and title
    axes[i].legend()
    axes[i].set_title(f'Feature {i}')
    axes[i].set_yticks([])

plt.tight_layout()  # Adjust layout to prevent overlap


In [ ]:

num_plots=10
idx=np.random.choice(np.arange(X.shape[1]),size=50000).astype(int)
fig, axes = plt.subplots(1, num_plots, figsize=(num_plots*4, 4))
for i in range(num_plots):
    axes[i].hist(Xtilde[:, i], bins=30, alpha=0.5, label=f'sampled', color='blue',density=True)
    axes[i].hist(X[:, i], alpha=0.5, label=f'true', color='orange',density=True)


    # Add legend and title
    axes[i].legend()
    axes[i].set_title(f'Feature {i}')
    axes[i].set_yticks([])

plt.tight_layout()  # Adjust layout to prevent overlap


In [ ]:
est = ProximalDE(cv=3, semi=True, n_jobs=-1, random_state=3, verbose=3, binary_D=False)
est.fit(Wtilde, Dtilde, Ztilde, Xtilde, Ytilde)
est.summary()

In [ ]:
import statsmodels.api as stm
def exp_res(it, generator, n, a, b, c, g, *, sy=1.0, n_jobs=-1, verbose=0):
    np.random.seed(it)

    # M is unobserved so we omit it from the return variables
    W, D, M, Z, X, Y = generator.sample(n, a, b, c, g, sy=sy, replace=True)

    res = stm.OLS(Y, np.hstack([D.reshape(-1, 1), M, X, np.ones((D.shape[0], 1))])).fit(cov_type='HC1')
    return res.params[0], np.sqrt(res.cov_params()[0, 0])

In [ ]:
D.mean(), Dtilde.mean()

In [ ]:
exp_res(5, generator, nsamples, a, b, c, g, n_jobs=-1)

In [ ]:
results = Parallel(n_jobs=-1, verbose=3)(delayed(exp_res)(i, generator, nsamples,
                                                          a, b, c, g, n_jobs=1)
                                          for i in range(100))

In [ ]:
points, stderrs = map(np.array, zip(*results))

print("Estimation Quality")
coverage = np.mean((points + 1.96 * stderrs >= c) & (points - 1.96 * stderrs <= c))
rmse = np.sqrt(np.mean((points - c)**2))
bias = np.abs(np.mean(points) - c)
std = np.std(points)
mean_stderr = np.mean(stderrs)
mean_length = np.mean(2 * 1.96 * stderrs)
median_length = np.median(2 * 1.96 * stderrs)
print(f"Coverage: {coverage:.3f}")
print(f"RMSE: {rmse:.3f}")
print(f"Bias: {bias:.3f}")
print(f"Std: {std:.3f}")
print(f"Mean CI length: {mean_length:.3f}")
print(f"Median CI length: {mean_length:.3f}")
print(f"Mean Estimated Stderr: {mean_stderr:.3f}")

In [ ]:
def exp_res(it, generator, n, a, b, c, g, *, sy=1.0,n_splits=3, semi=True,
            n_jobs=-1, verbose=0):
    np.random.seed(it)

    # M is unobserved so we omit it from the return variables
    W, D, _, Z, X, Y = generator.sample(n, a, b, c, g, sy=sy, replace=True)

    est = ProximalDE(cv=n_splits, semi=semi, binary_D=False,
                     n_jobs=n_jobs, random_state=it, verbose=verbose)
    est.fit(W, D, Z, X, Y)
    weakiv_stat, _, _, weakiv_crit = est.weakiv_test(alpha=0.05)
    idstr, _, _, idstr_crit = est.idstrength_violation_test(alpha=0.05)
    pval, _, _, pval_crit = est.primal_violation_test(alpha=0.05)
    dval, _, _, dval_crit = est.dual_violation_test(alpha=0.05)
    lb, ub = est.robust_conf_int(lb=-2, ub=2)
    return est.point_, est.stderr_, est.r2D_, est.r2Z_, est.r2X_, est.r2Y_, \
        idstr, idstr_crit, est.point_pre_, est.stderr_pre_, \
        pval, pval_crit, dval, dval_crit, weakiv_stat, weakiv_crit, \
        lb, ub

def exp_res(it, generator, n, a, b, c, g, *, sy=1.0,n_splits=3, semi=True,
            n_jobs=-1, verbose=0):
    np.random.seed(it)

    # M is unobserved so we omit it from the return variables
    W, D, _, Z, X, Y = generator.sample(n, a, b, c, g, sy=sy, replace=True)
    est = ProximalDE(cv=3, semi=True, n_jobs=-1, random_state=3, verbose=3, binary_D=False)
    est.fit(W, D, Z, X, Y)
    weakiv_stat, _, _, weakiv_crit = est.weakiv_test(alpha=0.05)
    idstr, _, _, idstr_crit = est.idstrength_violation_test(alpha=0.05)
    pval, _, _, pval_crit = est.primal_violation_test(alpha=0.05)
    dval, _, _, dval_crit = est.dual_violation_test(alpha=0.05)
    lb, ub = est.robust_conf_int(lb=-2, ub=2)
    return est.point_, est.stderr_, est.r2D_, est.r2Z_, est.r2X_, est.r2Y_, \
        idstr, idstr_crit, est.point_pre_, est.stderr_pre_, \
        pval, pval_crit, dval, dval_crit, weakiv_stat, weakiv_crit, \
        lb, ub

In [ ]:
results = []
from tqdm.auto import tqdm
for i in tqdm(range(100)):
    results.append(exp_res(i,generator,nsamples,a,b,c,g))


In [ ]:
import pickle as pk
pk.dump(results,open('results.pkl', 'wb'))

In [ ]:
# results = Parallel(n_jobs=-1, verbose=3)(delayed(exp_res)(i, generator, nsamples,
#                                                           a, b, c, g,
#                                                           n_splits=3, semi=True, n_jobs=1)
#                                           for i in range(100))

In [ ]:
points_base, stderrs_base, rmseD, rmseZ, rmseX, rmseY, \
    idstr, idstr_crit, points_alt, stderrs_alt, \
    pval, pval_crit, dval, dval_crit, wiv_stat, wiv_crit, \
    rlb, rub = map(np.array, zip(*results))

points_base = np.array(points_base)
stderrs_base = np.array(stderrs_base)
points_alt = np.array(points_alt)
stderrs_alt = np.array(stderrs_alt)

print("Estimation Quality")
for name, points, stderrs in [('Debiased', points_base, stderrs_base), ('Regularized', points_alt, stderrs_alt)]:
    print(f"\n{name} Estimate")
    coverage = np.mean((points + 1.96 * stderrs >= c) & (points - 1.96 * stderrs <= c))
    rmse = np.sqrt(np.mean((points - c)**2))
    bias = np.abs(np.mean(points) - c)
    std = np.std(points)
    mean_stderr = np.mean(stderrs)
    mean_length = np.mean(2 * 1.96 * stderrs)
    median_length = np.median(2 * 1.96 * stderrs)
    print(f"Coverage: {coverage:.3f}")
    print(f"RMSE: {rmse:.3f}")
    print(f"Bias: {bias:.3f}")
    print(f"Std: {std:.3f}")
    print(f"Mean CI length: {mean_length:.3f}")
    print(f"Median CI length: {mean_length:.3f}")
    print(f"Mean Estimated Stderr: {mean_stderr:.3f}")
    print(f"Nuisance R^2 (D, Z, X, Y): {np.mean(rmseD):.3f}, {np.mean(rmseZ):.3f}, {np.mean(rmseX):.3f}, {np.mean(rmseY):.3f}")

print("\nRobust ConfInt Coverage")
rcoverage = np.mean((rub >= c) & (rlb <= c))
print(f"Robust Coverage: {rcoverage:.3f}")

print("\nViolations")
for name, stat, crit in [('Id-Strenth', idstr, idstr_crit), ('WeakIV F-test', wiv_stat, wiv_crit)]:
    violation = np.mean(stat <= crit)
    print(f"% Violations of {name}: {violation:.3f}")
for name, stat, crit in [('Primal Existence', pval, pval_crit), ('Dual Existence', dval, dval_crit)]:
    violation = np.mean(stat >= crit)
    print(f"% Violations of {name}: {violation:.3f}")

## Semi-Synthetic Generation

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats
from joblib import Parallel, delayed
from proximalde.proximal import ProximalDE, residualizeW, svd_critical_value
from proximalde.utilities import covariance
from proximalde.gen_data import gen_data_with_mediator_violations, gen_data_no_controls_discrete_m, gen_data_no_controls, gen_data_complex

In [ ]:
a = 1.0  # a*b is the indirect effect through mediator
b = 1.0
c = .5  # this is the direct effect we want to estimate
d = .0  # this can be zero; does not hurt
e = 1.0  # if the product of e*f is small, then we have a weak instrument
f = 1.0  # if the product of e*f is small, then we have a weak instrument
g = .0  # this can be zero; does not hurt

n = 100000
pw = 10
pz, px = 4, 4

In [ ]:
np.random.seed(124)
# W, D, _, Z, X, Y = gen_data_with_mediator_violations(n, pw, pz, px, a, b, c, d, e, f, g)
# W = None

# W, D, _, Z, X, Y = gen_data_complex(n, pw, pz, px, a, b, c, d, e, f, g)

## for no controls un-comment this
# _, D, _, Z, X, Y = gen_data_no_controls(n, pw, pz, px, a, b, c, d, e, f, g)
# W = None

## for multi-dimensional mediator uncomment this
pm = 2
full_rank = False
while not full_rank:
    E = np.random.normal(0, 2, (pm, pz))
    F = np.random.normal(0, 2, (pm, px))
    if (np.linalg.matrix_rank(E, tol=0.5) == pm) and (np.linalg.matrix_rank(F, tol=0.5) == pm):
        full_rank = True
W, D, _, Z, X, Y = gen_data_no_controls_discrete_m(n, pw, pz, px, a, b, c, d, e*E, f*F, g, pm=pm)
W = None

In [ ]:
from proximalde.gen_data import SemiSyntheticGenerator

a = 1.0  # a*b is the indirect effect through mediator
b = 1.0
c = .5  # this is the direct effect we want to estimate
g = .0  # this can be zero; does not hurt
sm = 2.0  # strength of mediator noise; needs to be non-zero for identifiability; only used when pm=1.
nsamples = 100000

generator = SemiSyntheticGenerator(split=True)
generator.fit(W, D, Z, X, Y)

In [ ]:
import seaborn as sns 
sns.histplot(generator.Xepsilon_)

In [ ]:
Wtilde, Dtilde, _, Ztilde, Xtilde, Ytilde = generator.sample(nsamples, a, b, c, g, replace=True)

In [ ]:
sns.heatmap(covariance(Z, X))
plt.show()
sns.heatmap(covariance(Ztilde, Xtilde))

In [ ]:
sns.heatmap(covariance(Z, Z))
plt.show()
sns.heatmap(covariance(Ztilde, Ztilde))


In [ ]:
plt.hist(Ztilde[:, 0], label='sampled')
plt.hist(Z[:, 0], label='true')
plt.show()

In [ ]:
import statsmodels.api as stm
def exp_res(it, generator, n, a, b, c, g, *, sy=1.0, n_jobs=-1, verbose=0):
    np.random.seed(it)

    # M is unobserved so we omit it from the return variables
    W, D, M, Z, X, Y = generator.sample(n, a, b, c, g, sy=sy, replace=True)

    res = stm.OLS(Y, np.hstack([D.reshape(-1, 1), M, X, np.ones((D.shape[0], 1))])).fit(cov_type='HC1')
    return res.params[0], np.sqrt(res.cov_params()[0, 0])

In [ ]:
exp_res(5, generator, nsamples, a, b, c, g, n_jobs=1)

In [ ]:
results = Parallel(n_jobs=-1, verbose=3)(delayed(exp_res)(i, generator, nsamples,
                                                          a, b, c, g, n_jobs=1)
                                          for i in range(100))

In [ ]:
points, stderrs = map(np.array, zip(*results))

print("Estimation Quality")
coverage = np.mean((points + 1.96 * stderrs >= c) & (points - 1.96 * stderrs <= c))
rmse = np.sqrt(np.mean((points - c)**2))
bias = np.abs(np.mean(points) - c)
std = np.std(points)
mean_stderr = np.mean(stderrs)
mean_length = np.mean(2 * 1.96 * stderrs)
median_length = np.median(2 * 1.96 * stderrs)
print(f"Coverage: {coverage:.3f}")
print(f"RMSE: {rmse:.3f}")
print(f"Bias: {bias:.3f}")
print(f"Std: {std:.3f}")
print(f"Mean CI length: {mean_length:.3f}")
print(f"Median CI length: {mean_length:.3f}")
print(f"Mean Estimated Stderr: {mean_stderr:.3f}")

In [ ]:
Wtilde, Dtilde, _, Ztilde, Xtilde, Ytilde = generator.sample(nsamples, a, b, c, g, replace=True)

# we find that the dual violation still exists, causing a slight bias (the true
# value we should recover is c)
est = ProximalDE(cv=3, semi=True, n_jobs=-1, random_state=3, verbose=3)
est.fit(Wtilde, Dtilde, Ztilde, Xtilde, Ytilde)
est.summary()

In [ ]:
def exp_res(it, generator, n, a, b, c, g, *, sy=1.0,
            dual_type='Z', ivreg_type='adv', n_splits=3, semi=True,
            n_jobs=-1, verbose=0):
    np.random.seed(it)

    # M is unobserved so we omit it from the return variables
    W, D, _, Z, X, Y = generator.sample(n, a, b, c, g, sy=sy, replace=True)

    est = ProximalDE(cv=n_splits, semi=semi,
                     dual_type=dual_type, ivreg_type=ivreg_type,
                     n_jobs=n_jobs, random_state=it, verbose=verbose)
    est.fit(W, D, Z, X, Y)
    weakiv_stat, _, _, weakiv_crit = est.weakiv_test(alpha=0.05)
    idstr, _, _, idstr_crit = est.idstrength_violation_test(alpha=0.05)
    pval, _, _, pval_crit = est.primal_violation_test(alpha=0.05)
    dval, _, _, dval_crit = est.dual_violation_test(alpha=0.05)
    lb, ub = est.robust_conf_int(lb=-2, ub=2)
    return est.point_, est.stderr_, est.r2D_, est.r2Z_, est.r2X_, est.r2Y_, \
        idstr, idstr_crit, est.point_pre_, est.stderr_pre_, \
        pval, pval_crit, dval, dval_crit, weakiv_stat, weakiv_crit, \
        lb, ub

In [ ]:
results = Parallel(n_jobs=-1, verbose=3)(delayed(exp_res)(i, generator, nsamples,
                                                          a, b, c, g,
                                                          dual_type='Z', ivreg_type='adv',
                                                          n_splits=3, semi=True, n_jobs=1)
                                          for i in range(100))

In [ ]:
points_base, stderrs_base, rmseD, rmseZ, rmseX, rmseY, \
    idstr, idstr_crit, points_alt, stderrs_alt, \
    pval, pval_crit, dval, dval_crit, wiv_stat, wiv_crit, \
    rlb, rub = map(np.array, zip(*results))

points_base = np.array(points_base)
stderrs_base = np.array(stderrs_base)
points_alt = np.array(points_alt)
stderrs_alt = np.array(stderrs_alt)

print("Estimation Quality")
for name, points, stderrs in [('Debiased', points_base, stderrs_base), ('Regularized', points_alt, stderrs_alt)]:
    print(f"\n{name} Estimate")
    coverage = np.mean((points + 1.96 * stderrs >= c) & (points - 1.96 * stderrs <= c))
    rmse = np.sqrt(np.mean((points - c)**2))
    bias = np.abs(np.mean(points) - c)
    std = np.std(points)
    mean_stderr = np.mean(stderrs)
    mean_length = np.mean(2 * 1.96 * stderrs)
    median_length = np.median(2 * 1.96 * stderrs)
    print(f"Coverage: {coverage:.3f}")
    print(f"RMSE: {rmse:.3f}")
    print(f"Bias: {bias:.3f}")
    print(f"Std: {std:.3f}")
    print(f"Mean CI length: {mean_length:.3f}")
    print(f"Median CI length: {mean_length:.3f}")
    print(f"Mean Estimated Stderr: {mean_stderr:.3f}")
    print(f"Nuisance R^2 (D, Z, X, Y): {np.mean(rmseD):.3f}, {np.mean(rmseZ):.3f}, {np.mean(rmseX):.3f}, {np.mean(rmseY):.3f}")

print("\nRobust ConfInt Coverage")
rcoverage = np.mean((rub >= c) & (rlb <= c))
print(f"Robust Coverage: {rcoverage:.3f}")

print("\nViolations")
for name, stat, crit in [('Id-Strenth', idstr, idstr_crit), ('WeakIV F-test', wiv_stat, wiv_crit)]:
    violation = np.mean(stat <= crit)
    print(f"% Violations of {name}: {violation:.3f}")
for name, stat, crit in [('Primal Existence', pval, pval_crit), ('Dual Existence', dval, dval_crit)]:
    violation = np.mean(stat >= crit)
    print(f"% Violations of {name}: {violation:.3f}")